<a href="https://colab.research.google.com/github/WeiZhenLim/2X/blob/main/AirTable_LinkedIn_Data_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### AirTable LinkedIn Data Transfer
---

***NOTE:***
1. Only update the section with "<font color='orange'>Update variables</font>".
2. Check the output for each line of code to ensure that the number of instances/rows exported and compiled matches LinkedIn Campaign Manager.
3. If you encounter any errors, you can either try to resolve them yourself or reach out to Wei Zhen Lim.
4. Any edits or changes made will not affect the source code. Feel free to edit the code.

***Workflow:***
1. Read Ads Performance Reports.
2. Read Ads Bulk Export.
3. Obtain launch and completion dates from Ads Performance Reports (Table 1).
4. Preprocess Ads Performance Report (Table 2).
5. Preprocess Ads Bulk Export (Table 3).
6. Merge all three tables from steps 3, 4, and 5. Export the output as preview (Table 4).
7. Preprocess the merged table (Table 4) to match AirTable's column/field format.


### Run the following code without editing (Setting Up Environment)

In [18]:
# Clone repo into Colab & Add Path
!git clone https://github.com/WeiZhenLim/2X
%ls

import sys
sys.path.append("/content/2X/python_project_2x")

fatal: destination path '2X' already exists and is not an empty directory.
2X/  drive/  sample_data/


In [19]:
from csv_hacks import compile_csv
from airtable_data_transfer.linkedin_transfer import *

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### <font color='orange'>Update Variables</font>
---
1. Create a new folder and save all the Ads Performance Reports (This shall be the <font color='red'>**Google Drive path**</font> for `folder_path_report`)
2. Create a new folder and save all the Ads Bulk Export (This shall be the <font color='red'>**Google Drive path**</font> for `folder_path_bulk`)
3. `dest_folder` = <font color='red'>**Google Drive path**</font> where you wish to save all the output generated from this script.

In [21]:
# Folder path containing all the Ads Performance Report(s)
folder_path_report = r"/content/drive/MyDrive/2X - General/03 AirTable LinkedIn Data Transfer/Ad Performance Export Example"

In [22]:
# Folder path containing all the Ads Bulk Export(s)
folder_path_bulk = r"/content/drive/MyDrive/2X - General/03 AirTable LinkedIn Data Transfer/Ads Bulk Export Example"

In [23]:
# Destination folder path to save the output
dest_folder = r"/content/drive/MyDrive/2X - General/03 AirTable LinkedIn Data Transfer"

### 1. Read Ads Performance Reports

In [24]:
df_report = compile_csv(folder_path_report, dest_folder, "ads_performance_report", "Start Date (in UTC)")

01 Ads Performance Report Jan 2023 - May 2023.csv
Skipping 5 rows
>>> 01 Ads Performance Report Jan 2023 - May 2023.csv contains 176 rows and 94 columns.

02 Ads Performance Report June 2023 - Nov 2023.csv
Skipping 5 rows
>>> 02 Ads Performance Report June 2023 - Nov 2023.csv contains 596 rows and 96 columns.

03 Ads Performance Report Dec 2023 - 25th Feb 2024.csv
Skipping 5 rows
>>> 03 Ads Performance Report Dec 2023 - 25th Feb 2024.csv contains 809 rows and 74 columns.

>>>> Successfully compiled. Total no of rows = 1581 <<<<


### 2. Read Ads Bulk Export

In [25]:
df_bulk = compile_csv(folder_path_bulk, dest_folder, "ads_bulk_export", "*Account ID")

Ads Bulk Export Example.csv
Skipping 4 rows
>>> Ads Bulk Export Example.csv contains 82 rows and 15 columns.

>>>> Successfully compiled. Total no of rows = 82 <<<<


### 3. Obtain launch and completion dates from Ads Performance Report (Table 1)

In [26]:
df_launch_completed_date = get_launch_completed_date(df_report)

There are 391 instances with zero clicks. Original data: 1581 instances; Filtered data: 1190 instances.


### 4. Preprocess Ads Performance Reports (Table 2)

In [27]:
df_report = preprocess_ads_performance_report(df_report)

### 5. Preprocess Ads Bulk Export (Table 3)

In [28]:
df_bulk = preprocess_ads_bulk_report(df_bulk)

### 6. Merge all three tables from steps 3, 4, and 5. Export the output as preview (Table 4)

In [29]:
df_final = merge_and_preprocess(df_launch_completed_date, df_report, df_bulk)

In [30]:
check_export_count(df_final)

Campaign Group ID    14
Campaign ID          18
Ad ID                51
dtype: int64

In [31]:
# Preview File
date_code_execute = datetime.today().strftime("%Y%m%d")
df_final.to_csv(os.path.join(dest_folder, date_code_execute + "-Preview.csv"), index=False)

### Download the Column Mapping CSV File

In [32]:
from google.colab import files

files.download("/content/2X/python_project_2x/airtable_data_transfer/AirTable Column Mapping.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### <font color='orange'>Update Variables</font>
---
1. `mapping_file` = <font color='red'>**Google Drive path**</font> for the AirTable Column Mapping CSV file (Ensure that the column names match those in the AirTable Column CSV file).
2. `airtable_col_file` = <font color='red'>**Google Drive path**</font> for the AirTable Column CSV file.

In [33]:
# CSV File for Column Mapping
mapping_file = r"/content/drive/MyDrive/2X - General/03 AirTable LinkedIn Data Transfer/AirTable Column Mapping.csv"

In [34]:
# AirTable Columns CSV File
airtable_col_file = r"/content/drive/MyDrive/2X - General/03 AirTable LinkedIn Data Transfer/AirTable Column.csv"

### 7. Preprocess the merged table (Table 4) to match AirTable's column/field format.

In [35]:
airtable_mapping(mapping_file, df_final, airtable_col_file, dest_folder, "Final Output")